In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def parse_output_file(file_path):
    with open(file=file_path) as f:
        lines = f.readlines()
        f.close()
    passed_header = False
    times, names = [], []
    for line in lines:
        l = line.split()
        if l:
            if l[0] == "ncalls":
                passed_header = True
            else:
                if passed_header:
                    times.append(float(l[3]))
                    name = ""
                    name_list = l[5:]
                    for i, name_part in enumerate(name_list):
                        name += name_part
                        if not i == len(name_list) - 1:
                            name += " "
                    names.append(name)
    # create DataFrame
    data = pd.DataFrame({
        "percentage (of {} sec)".format(times[0]): 100 * np.array(times) / times[0], 
        "module / function": names
    })
    return data

In [3]:
def get_avg_data(directory):
    data_list = []
    for filename in os.listdir(directory):
        if filename.startswith("output_") and filename.endswith(".txt"):
            if filename == "output_sdf_master.txt":
                pass
            else:
                f = os.path.join(directory, filename)
                data_list.append(parse_output_file(f))

    # calculate averages
    times, names = [], []
    for name in data_list[0]['module / function'].values:
        names.append(name)
        t, count = 0.0, 0
        for data in data_list:
            d = data.loc[data['module / function'] == name]
            if list(d[d.columns[0]].values):
                t += float(d[d.columns[0]].values[0])
                count += 1
        times.append(t / float(count))
        
    t_total = 0.0
    for data in data_list:
        t_total += float(data.columns[0].split()[-2])
    t_total = t_total / len(data_list)
        
    t_print = ""
    t_hour = t_total / 3600
    if t_hour <= 1.0:
        t_min = 60 * t_hour
        if t_min <= 1.0:
            t_print = "{:.2f} sec".format(60 * t_min)
        else:
            t_print = "{:.2f} min".format(t_min)
    else:
        t_print = "{:.2f} hours".format(t_hour)

    avg_data = pd.DataFrame({
        "% of {}".format(t_print): np.array(times), 
        "module / function": names
    })
    return avg_data

In [4]:
cpu_data = get_avg_data("./2023-05-11_BO/CustomMean_cpu/")
gpu_data = get_avg_data("./2023-05-11_BO/CustomMean_gpu/")

In [5]:
data = pd.concat([cpu_data, gpu_data], axis=1)
data.columns=[['CPU','CPU','GPU','GPU'], list(data.columns)]

In [6]:
data[:50]

CPU                                                      
   % of 3.45 hours                                  module / function   
0       100.000000                    {built-in method builtins.exec}  \
1       100.000000                        run_bo_agent.py:1(<module>)   
2        98.809184                               bo_agent.py:157(run)   
3        73.851022               timeout.py:17(minimize_with_timeout)   
4        73.850840                          _minimize.py:45(minimize)   
5        73.848391                _lbfgsb_py.py:210(_minimize_lbfgsb)   
6        73.339233     _differentiable_functions.py:282(fun_and_grad)   
7        73.273978                _optimize.py:67(_compute_if_needed)   
8        73.236810      _differentiable_functions.py:249(_update_fun)   
9        73.230035       _differentiable_functions.py:154(update_fun)   
10       73.224176      _differentiable_functions.py:132(fun_wrapped)   
11       73.171276                          _optimize.py:74(__call__)   
12       73.251453                     optimize.py:415(optimize_acqf)   
13       73.251260                    optimize.py:547(_optimize_acqf)   
14       73.251191              optimize.py:258(_optimize_acqf_batch)   
15       72.982664        optimize.py:294(_optimize_batch_candidates)   
16       72.978909                    gen.py:43(gen_candidates_scipy)   
17       71.681752                           gen.py:175(f_np_wrapper)   
18       65.374202                         module.py:1494(_call_impl)   
19       60.991418                         gpytorch.py:564(posterior)   
20       64.295041                                  base.py:188(step)   
21       64.167542                 bayesian_generator.py:67(generate)   
22       57.332995                          exact_gp.py:251(__call__)   
23       57.133806                         model_list.py:87(__call__)   
24       57.040649                       model_list.py:88(<listcomp>)   
25       62.763287           bayesian_generator.py:147(optimize_acqf)   
26       55.369865                             module.py:30(__call__)   
27       52.023743                      gp_regression.py:174(forward)   
28       51.749190                               mean.py:17(__call__)   
29       50.800837                         custom_mean.py:26(forward)   
30       50.757243                              utils.py:148(forward)   
31       50.874135                          prior_mean.py:28(forward)   
32       50.114192                              module.py:50(forward)   
33       50.025950            transformed_model.py:49(evaluate_model)   
34       48.820319                       module.py:41(evaluate_model)   
35       48.787970                             model.py:118(evaluate)   
36       43.741023                          container.py:215(forward)   
37       41.242846                       transforms.py:326(decorated)   
38       41.234683                       transforms.py:266(decorated)   
39       41.011995              constrained_acqusition.py:72(forward)   
40       41.027485                                      gen.py:229(f)   
41       35.060708                        monte_carlo.py:164(forward)   
42       29.058124                             linear.py:113(forward)   
43       28.738026              {built-in method torch._C._nn.linear}   
44       30.291307  {method 'run_backward' of 'torch._C._EngineBas...   
45       30.100294                              __init__.py:204(grad)   
46       24.004346                 bo_agent.py:96(_calculate_metrics)   
47       12.645349                         activation.py:516(forward)   
48       12.554112                            functional.py:1538(elu)   
49       12.424359                 {built-in method torch._C._nn.elu}   

               GPU                                                     
   % of 1.48 hours                                  module / function  
0       100.000000                    {built-in method builtins.exec}  
1 